In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras as K
import skimage
from skimage.io import imshow, imread, imsave
from glob import glob
import os


In [3]:
import random

In [ ]:
#Setting variables to hold the input and output directory paths
in_dir = '/content/drive/My Drive//Part_B_Assignment_1/Original'
out_dir = '/content/drive/My Drive//Part_B_Assignment_1/Kmeans'

#Read the input image file names with paths into a list
in_files = in_dir + '/*.png' #all files with .png extension will be selected
img_names = glob(in_files) #glob returns a list of file names with paths that match the in_files pattern
img_names= random.sample(img_names, 1000)
#print(img_names) #prints the list of files names
len(img_names)

1000

In [ ]:
for fn in img_names:
    print('processing %s...' % fn)

    img = cv2.imread(fn)
    img2 = img.reshape((-1,3)) #reshaping the image into a 2D array, each column representing a RBG colour channel for each pixel
    img2=np.float32(img2) #converts image into float32, needed for kmeans
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) #defining the stopping criteria for the kmeans algorithm
                #the criteria include epsilon set at 1.0, and maximum iterations set at 10
    k=3
    attempts=10 #10 iterations, with different starting centres, best result is chosen
    ret,label,center=cv2.kmeans(img2,k,None,criteria,attempts,cv2.KMEANS_PP_CENTERS) #ret: how tight the clusters are packed, label: the cluster label for each pixel, center:final cluster center
    #Reconstructing the segmented image
    center=np.uint8(center) #reconverting the center back into an 8bit integer
    res = center[label.flatten()] #converts the 2D array image into a 1D array, then replaces each pixel with the colour of its assigned cluster center
    res2 = res.reshape((img.shape)) #reshapes output image into the original shape


    # write the result to disk in the previously created output directory
    name = os.path.basename(fn)
    outfile = out_dir + '/' + name
    cv2.imwrite(outfile, res2)

processing /content/drive/My Drive//Part_B_Assignment_1/Original/b4c52960-bbee-439e-83dc-f89ddaade71b.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/d75933ed-d26e-4d1f-bda3-96bc0d77fb36.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/ebb53337-1d4f-4ab5-9b6d-83a66f48fe2e.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/dec61ae1-5037-4c3a-8952-15371becca7d.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/b59dc3a0-3d54-4e3b-a485-33b3e43966de.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/2dbf6f69-e020-4948-9e49-4b33edb7cebd.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/71881a39-a30c-40c7-8c4a-9f64c93e79d9.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/b87fb62d-2c36-4f68-aba9-37fcd7d46eef.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Original/44b4a765-104e-4182-8871-e933c83c91ef.png...
processing /content

In [4]:
in_dir2 = '/content/drive/My Drive//Part_B_Assignment_1/Kmeans'

out_dir2 = '/content/drive/My Drive//Part_B_Assignment_1/Kmeans_mask'

# read the input image file names with paths into a list
infiles2 = in_dir2 + '/*.png'
img_names2 = glob(infiles2)
#print(img_names2)
len(img_names2)

821

In [5]:
for fn in img_names2:
    print('processing %s...' % fn)

    # read an input image as gray
    im_gray = cv2.imread(fn,0)

    #blur = cv2.GaussianBlur(im_gray,(5,5),0)

    # threshold it with your saved threshold
    im_bw = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # write the result to disk in the previously created output directory
    name = os.path.basename(fn)
    outfile = out_dir2 + '/' + name
    cv2.imwrite(outfile, im_bw)

processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/b4c52960-bbee-439e-83dc-f89ddaade71b.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/d75933ed-d26e-4d1f-bda3-96bc0d77fb36.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/ebb53337-1d4f-4ab5-9b6d-83a66f48fe2e.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/dec61ae1-5037-4c3a-8952-15371becca7d.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/b59dc3a0-3d54-4e3b-a485-33b3e43966de.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/2dbf6f69-e020-4948-9e49-4b33edb7cebd.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/71881a39-a30c-40c7-8c4a-9f64c93e79d9.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/b87fb62d-2c36-4f68-aba9-37fcd7d46eef.png...
processing /content/drive/My Drive//Part_B_Assignment_1/Kmeans/44b4a765-104e-4182-8871-e933c83c91ef.png...
processing /content/drive/My Drive//P

In [6]:
dice_total=0.0
jacc_total=0.0
image_count=0

In [7]:
#Calculating dice coefficient

def dice_coef2(a, b):

    hi = a/255 #dividing by 255 so that the values are in the range [0,1] -standardising
    pred = (255-b)/255

    hi= K.backend.flatten(hi) #flattening the pixel values into a 1D array

    pred= K.backend.flatten(pred)

    numerator= 2 * K.backend.sum(hi*pred)

    denominator= K.backend.sum(hi+pred)
    dice = numerator / denominator

    return dice


In [8]:
img_names2.sort()
out_files2 =  out_dir2 + '/*.png'
out_names2 = glob(out_files2)
out_names2.sort()

In [9]:
# Iterate through the original and segmented image pairs
for gt_img_name, seg_img_name in zip(img_names2, out_names2):
    # Ensure the file names match
    if os.path.basename(gt_img_name) == os.path.basename(seg_img_name):
        # Read the original and segmented images in grayscale
        ground_truth = cv2.imread(gt_img_name, 0)
        segmented_img = cv2.imread(seg_img_name, 0)

        # Check if the images were loaded correctly
        if ground_truth is None or segmented_img is None:
            print(f"Error loading images: {gt_img_name} or {seg_img_name}")
            continue

        # Calculate Dice Coefficient for the current pair of images
        dice_score = dice_coef2(ground_truth, segmented_img)
        jacc_score = dice_score / (2-dice_score)
        # Convert TensorFlow tensor to a numerical value (using numpy)
        dice_value = dice_score.numpy()
        jacc_value = jacc_score.numpy()
        # Accumulate the Dice score
        dice_total += dice_value
        jacc_total += jacc_value
        image_count += 1

        # Print the Dice Coefficient for the current image
        print(f"Dice Coefficient for {os.path.basename(gt_img_name)}: {dice_value}")
    else:
        print(f"File mismatch: {os.path.basename(gt_img_name)} and {os.path.basename(seg_img_name)}")

# Calculate and print the average Dice score
if image_count > 0:
    average_dice_score = dice_total / image_count
    average_jacc_score = jacc_total / image_count
    print(f"\nAverage Dice Coefficient for the entire dataset: {average_dice_score}")
    print(f"\nAverage Jaccard Coefficient for the entire dataset: {average_jacc_score}")
else:
    print("No matching images found.")

Dice Coefficient for 003a89b0-a095-417a-8dd6-f408339bbc68.png: 0.8592464350797053
Dice Coefficient for 003d1cdc-0aec-430b-90e0-f4d82135b1ac.png: 0.921263239535164
Dice Coefficient for 005e60b6-77b8-458c-b57c-bfe0c7e7df78.png: 0.8634248530147073
Dice Coefficient for 00a02700-2ea2-4590-9e15-ffc9160fd3de.png: 0.884444160376605
Dice Coefficient for 00c8364b-8c85-4502-bcfe-64736fe76815.png: 0.8740238610142265
Dice Coefficient for 00d04a90-80e5-4bce-9511-1b64eabb7a47.png: 0.8231228989044157
Dice Coefficient for 01b38a56-13eb-447b-89ee-fc21cb40dcc1.png: 0.8357944347169128
Dice Coefficient for 01b99da8-b66b-464b-a7ee-1cadab8db18b.png: 0.8627335722183969
Dice Coefficient for 027ab174-e47a-44f8-b85b-372449eeb7d8.png: 0.9114583049517218
Dice Coefficient for 02fb5e19-6f94-4787-bb78-e550a29a3cbd.png: 0.8914711351166548
Dice Coefficient for 030f3421-ed3f-4bf1-8f27-4d02590bffbc.png: 0.873405246345306
Dice Coefficient for 035f6c4b-0897-46ab-9c3b-5f3b0feff043.png: 0.8892642526238005
Dice Coefficient fo